<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/template/template.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Clasificación supervisada SCART
Script trabajado por Francisco Albornoz (2021-04-28) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [1]:
# Installs geemap package
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print("Installing geemap ...")
#     subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

In [2]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Crear un mapa interactivo 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Escogemos la imagen landsat 

In [4]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")


## Importar rasters locales 

In [5]:
%pwd

'/mnt/d/Francisco Albornoz/Github/VitoAlbornoz/GEE'

In [6]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 
# os.path.expanduser('/mnt/c/Users/Usuario/Documents/Github/Vitoalbornoz/')

In [7]:
# verificar en el sistema del pc cual es la ruta
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [8]:
out_dir

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data'

In [9]:
raster_aysen

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif'

In [10]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')
Map

The image file does not exist.


Map(center=[-44, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [11]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [12]:
# raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [13]:
# raster.bounds

In [14]:
# raster.crs

In [15]:
# raster

In [16]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")

landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")

Map

Map(center=[-44, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Obteniendo los valores para el Training Datasets

In [17]:
# points = mosaico.sample(**{
# #     'region': landsat.geometry(),
#     'scale': 30,
#     'numPixels': 20000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })

# Map.addLayer(points, {}, 'training', False)
# Map

In [18]:
work_dir = os.path.expanduser('../Data/shapes/')

merged_shp = os.path.join(work_dir, 'roi_h.shp')
val_shp= os.path.join(work_dir, 'validacion.shp')
merged_original =os.path.join(work_dir,'roi_merged.shp')

In [19]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [20]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [21]:
mergedoriginal =geemap.shp_to_ee(merged_original)
Map.addLayer(mergedoriginal, {}, 'Áreas de entrenamiento Original')

In [22]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['IDH'],
    'scale' : 30 ,
})

In [23]:
datasetval = mosaico.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [24]:
dataset1 = mosaico.sampleRegions(**{
    'collection': mergedoriginal,
    'properties': ['ID'],
    'scale': 30,
})

In [25]:
print(dataset1.size().getInfo())

14254


In [26]:
print(datasetval.size().getInfo())

3097


In [27]:
dataset.propertyNames()

In [28]:
datasetval.propertyNames()

In [29]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'IDH': 9, 'b1': 0.026168961077928543, 'b2': 0.04253531992435455, 'b3': 0.03470490872859955, 'b4': 0.33766427636146545, 'b5': 0.1495322287082672, 'b6': 0.06873124837875366, 'b7': 0.8135994672775269, 'b8': 0.7762474417686462, 'b9': 0.5209251642227173}}


In [30]:
print(datasetval.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'ID': 3, 'b1': 0.01979999989271164, 'b2': 0.017999999225139618, 'b3': 0.010499999858438969, 'b4': 0.01730000041425228, 'b5': 0.00989999994635582, 'b6': 0.0071000000461936, 'b7': 0.24460433423519135, 'b8': -0.019829994067549706, 'b9': 0.019325504079461098}}


In [31]:
# out_csv = os.path.join(out_dir, 'Dataset_1.csv')
# geemap.extract_values_to_points(merged, mosaico, out_csv)

### Dividiendo en muestras y datos de validación

In [32]:
# Guardamos las bandas
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

# Esta propiedad guarda las clases de landcover
label = 'IDH'
label1 ='ID'
# Overlay the points on the imagery to get training.
# sample = landsat.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.2

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [33]:
dataset.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'IDH': 9,
  'b1': 0.026168961077928543,
  'b2': 0.04253531992435455,
  'b3': 0.03470490872859955,
  'b4': 0.33766427636146545,
  'b5': 0.1495322287082672,
  'b6': 0.06873124837875366,
  'b7': 0.8135994672775269,
  'b8': 0.7762474417686462,
  'b9': 0.5209251642227173}}

In [34]:
val.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-72.31311015654404, -45.68851546739174],
    [-72.31311710746614, -45.68847354279539],
    [-72.31314220327586, -45.68843517553851],
    [-72.3131829873927, -45.68840412126554],
    [-72.31323546757821, -45.68838341977111],
    [-72.31329450672591, -45.688375097449025],
    [-72.31335432570957, -45.68837996893968],
    [-72.31340906907025, -45.68839755738974],
    [-72.31345337817417, -45.688426141129185],
    [-72.31348291574247, -45.688462922196415],
    [-72.31349479041245, -45.68850430021699],
    [-72.31348783977175, -45.68854622482846],
    [-72.31346274415755, -45.68858459215558],
    [-72.31342196007567, -45.688615646527104],
    [-72.3133694797512, -45.68863634811076],
    [-72.31331044034371, -45.68864467047863],
    [-72.31325062107865, -45.68863979897288],
    [-72.31319587752246, -45.68862221045255],
    [-72.31315156838357, -45.68859362661456],
    [-72.31312203095422, -45.68855684545811],
    [-72.3

## Entrenando el modelo 

In [35]:

trained = ee.Classifier.smileCart().train(dataset, label, bands)
classified = mosaico.select(bands).classify(trained)

In [36]:
trained1 = ee.Classifier.smileCart().train(dataset1, label1, bands)
classified1 = mosaico.select(bands).classify(trained1)

In [37]:
trained

In [38]:
classified

### Clasificando la imagen

In [39]:
# # Clasificando con las mismas bandas de entrenamiento
# # result = landsat.select(bands).classify(classified)
# class_palette = ['22c129',
#  'f0ff00',
#  '0c40f1',
#  '685205',
#  '0d4404',
#  'e82d13',
#  'f99504',
#  '17d4c6',
#  '1c6330',
#  '24ff00',
#  '2de50f',
#  'ccba7d',
#  'dec61a',
#  '28d909',
#  '03c2ff',
#  '4e8c44',
#  'fdfdfd',
#  'c0c233'
# ]
# # # Display the clusters with random colors.
# Map.addLayer(classified,
#              {'min': 0, 'max': 1, 'palette' : class_palette },
#              'CART')
# Map
             

In [40]:
# class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
#     ]

In [41]:
# land = classified.set('classification_class_values', class_values)
# landcoverf = land.set('classification_class_palette', class_palette)

In [42]:
# Map.addLayer(landcoverf, {}, 'Land cover SCART')
# Map

In [43]:
# Map.addLayer(classified,
#              {'min': 0, 'max': 11, 'palette': ['0c40f1', #1 agua*
#  'f0ff00', #2 cultivo*
#  'f99504', #3 estepa*
#  '17d4c6', #4 humedal*
#  '24ff00', #5 matorral abierto*
#  '2de50f', #6 matorral arborescente*
#  'ccba7d', #7 plantaciones*
#  'dec61a', #8 praderas*
#  '28d909', #9 primario*
#  '4e8c44', #10 secundario*
#  '685205', #11 suelo desnudo*

# ]},
#              'classification Smile Cart')
# Map

In [44]:
class_values = [1,2,3,4,5,6,7,8,9,10,11
    ]
class_values1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

In [45]:
class_palette = ['0c40f1', #1 agua*
 'f0ff00', #2 cultivo*
 'f99504', #3 estepa*
 '17d4c6', #4 humedal*
 '24ff00', #5 matorral abierto*
 '2de50f', #6 matorral arborescente*
 'ccba7d', #7 plantaciones*
 'dec61a', #8 praderas*
 '28d909', #9 primario*
 '4e8c44', #10 secundario*
 '685205', #11 suelo desnudo*
]
class_palette1 = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [46]:
land = classified.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)
Map.addLayer(landcoverf, {}, 'Land cover smile Cart')


In [47]:
land1 = classified1.set('classification_class_values', class_values1)
landcoverf1 = land1.set('classification_class_palette', class_palette1)
Map.addLayer(landcoverf1, {}, 'Land cover smile Cart Original')
Map

Map(center=[-44, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Categorizando según las clases 

In [48]:
# landcover.getInfo()

In [49]:
# class_values = [1,
#  2,
#  3,
#  4,
#  5,
#  6,
#  7,
#  8,
#  9,
#  10,
#  11,
#  12,
#  13,
#  14,
#  15,
#  16]

In [50]:
# class_palette = ['476ba1',
#  'd1defa',
#  'decaca',
#  'd99482',
#  'ee0000',
#  'ab0000',
#  'b3aea3',
#  '68ab63',
#  '1c6330',
#  'b5ca8f',
#  'a68c30',
#  'ccba7d',
#  'e3e3c2',
#  'caca78',
#  '99c247',
#  '78ae94']


In [51]:
# landcoverf = result.set('classification_class_values', class_values)
# landcoverf = landcoverf.set('classification_class_palette', class_palette)

In [52]:
# Map.addLayer(landcoverf, {}, 'Land cover clasificado' )
# Map

### Visualizando el resultado


In [53]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))


Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [54]:
Map

Map(center=[-44, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Agregando título al mapa

In [55]:
# Map.add_legend('Clasificación supervisada CART')
# Map

### Evaluación de presición / Accuracy assessment

In [56]:
train_accuracy = trained1.confusionMatrix()

In [57]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 625, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 806, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 3020, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1623, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 931, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 621, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 222, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 162, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3098, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 258, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 781, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 217, 0, 0, 0],
 [0, 0, 0

In [58]:
train_accuracy.accuracy().getInfo()

1

In [59]:
train_accuracy.kappa().getInfo()

1

In [60]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [61]:
train_accuracy.consumersAccuracy().getInfo()

[[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

#### Set de datos de validación

In [62]:
validated = datasetval.classify(trained1)

In [63]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'ID': 3,
  'b1': 0.01979999989271164,
  'b2': 0.017999999225139618,
  'b3': 0.010499999858438969,
  'b4': 0.01730000041425228,
  'b5': 0.00989999994635582,
  'b6': 0.0071000000461936,
  'b7': 0.24460433423519135,
  'b8': -0.019829994067549706,
  'b9': 0.019325504079461098,
  'classification': 3}}

In [64]:
test_accuracy = validated.errorMatrix('ID', 'classification')

In [65]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 [0, 0, 0, 30, 0, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 7, 0, 0, 0],
 [0, 0, 0, 0, 600, 0, 0, 2, 0, 0, 0, 0, 0, 18, 0, 2, 0, 0, 7],
 [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 37, 0, 0, 188, 0, 0, 0, 0, 0, 22, 0, 0, 0, 0, 91],
 [0, 0, 0, 1, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 8, 15, 0, 21, 14, 3, 2, 0, 10, 2, 18, 18, 13, 8, 0, 0, 3, 7],
 [0, 41, 21, 0, 0, 21, 10, 0, 0, 0, 2, 67, 17, 0, 64, 0, 17, 0, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 0, 5, 0, 0, 0, 0],
 [0, 0, 126, 0, 3, 7, 0, 5, 0, 9, 5, 27, 0, 56, 0, 0, 0, 0, 53],
 [0, 363, 39, 0, 8, 8, 22, 0, 0, 0, 0, 33, 10, 0, 378, 0, 227, 8, 4],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

In [66]:
test_accuracy.accuracy().getInfo()

0.4833710041976106

In [67]:
test_accuracy.kappa().getInfo()

0.4103942524386442

In [68]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.3333333333333333],
 [0.926829268292683],
 [0.5882352941176471],
 [0.9538950715421304],
 [1],
 [0],
 [0.5562130177514792],
 [0],
 [1],
 [0.014084507042253521],
 [0.250936329588015],
 [0.84375],
 [0.19243986254295534],
 [0.34363636363636363],
 [0],
 [0.6341463414634146],
 [0],
 [0]]

In [69]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.002325581395348837,
  0.15510204081632653,
  0.967741935483871,
  0.8645533141210374,
  0.07407407407407407,
  0,
  0.9543147208121827,
  0,
  0.09523809523809523,
  0.2222222222222222,
  0.4589041095890411,
  0.34177215189873417,
  0.5137614678899083,
  0.7777777777777778,
  0,
  0.29714285714285715,
  0,
  0]]

### Descargando la matrix

In [70]:
# import csv
# import os

# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
# training_csv = os.path.join(out_dir, 'train_accuracy.csv')
# testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(train_accuracy.getInfo())
    
# with open(testing_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(test_accuracy.getInfo())

### Exportando 

In [71]:
# import os
# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [72]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)


In [73]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)